In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

In [2]:
data_dir = '../input/optiver-realized-volatility-prediction/'

## Functions for preprocess

In [3]:
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap
def calc_wap3(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2']+ df['ask_size2'])
    return wap


def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def count_unique(series):
    return len(np.unique(series))

In [4]:
book_train = pd.read_parquet(data_dir + "book_train.parquet/stock_id=15")

## Main function for preprocessing book data

In [5]:
def preprocessor_book(file_path):
    df = pd.read_parquet(file_path)
    #calculate return etc
    df['wap'] = calc_wap(df)
    df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
    
    df['wap2'] = calc_wap2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    df['wap3'] = calc_wap3(df)
    df['log_return3'] = df.groupby('time_id')['wap3'].apply(log_return)
    
    df['wap_balance'] = abs(df['wap'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

    #dict for aggregate
    create_feature_dict = {
        'log_return':[realized_volatility],
        'log_return2':[realized_volatility],
        'log_return3':[realized_volatility],
        'wap_balance':[np.mean],
        'price_spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean],
        'wap':[np.mean],
            }

    #####groupby / all seconds
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
        
    ######groupby / last XX seconds
    last_seconds = [300]
    
    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    #create row_id
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)
    
    return df_feature

In [6]:
%%time
file_path = data_dir + "book_train.parquet/stock_id=0"
preprocessor_book(file_path)

CPU times: user 11.6 s, sys: 319 ms, total: 11.9 s
Wall time: 12 s


,log_return_realized_volatility,log_return2_realized_volatility,log_return3_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,...,log_return2_realized_volatility_300,log_return3_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,wap_mean_300,row_id
0,0.004499,0.006999,0.006119,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,...,0.004864,0.004492,0.000372,0.000822,0.000223,-0.000162,137.158273,294.928058,1.003753,0-5
1,0.001204,0.002476,0.002320,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,...,0.002009,0.001637,0.000239,0.000353,0.000164,-0.000123,135.513043,484.521739,1.000397,0-11
2,0.002369,0.004801,0.004684,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,...,0.003196,0.002354,0.000431,0.000689,0.000141,-0.000249,144.147059,455.235294,0.998685,0-16
3,0.002574,0.003637,0.002881,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,...,0.002713,0.001814,0.000331,0.000833,0.000158,-0.000095,144.698113,418.169811,0.998436,0-31
4,0.001894,0.003257,0.003166,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,...,0.002188,0.002443,0.000252,0.000425,0.000191,-0.000120,99.449438,407.584270,0.999488,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.002579,0.003821,0.004029,0.000212,0.000552,0.000083,-0.000182,197.144781,374.235690,0.997938,...,0.002573,0.002769,0.000193,0.000509,0.000062,-0.000169,233.946667,350.560000,0.997519,0-32751
3826,0.002206,0.002847,0.002848,0.000267,0.000542,0.000092,-0.000172,233.781553,621.131068,1.000310,...,0.002255,0.002007,0.000300,0.000588,0.000074,-0.000177,257.920000,668.640000,1.000682,0-32753
3827,0.002913,0.003266,0.003235,0.000237,0.000525,0.000202,-0.000083,115.829787,343.734043,0.999552,...,0.002646,0.002559,0.000216,0.000446,0.000191,-0.000075,105.432692,326.759615,1.000111,0-32758
3828,0.003046,0.005105,0.004688,0.000245,0.000480,0.000113,-0.000166,132.074919,385.429967,1.002357,...,0.003934,0.003472,0.000269,0.000516,0.000096,-0.000175,123.423313,394.588957,1.002277,0-32763


In [7]:
trade_train = pd.read_parquet(data_dir + "trade_train.parquet/stock_id=0")

## Main function for preprocessing trade data

In [8]:
def preprocessor_trade(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    
    aggregate_dictionary = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }
    
    df_feature = df.groupby('time_id').agg(aggregate_dictionary)
    
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    
    ######groupby / last XX seconds
    last_seconds = [300]
    
    for second in last_seconds:
        second = 600 - second
    
        df_feature_sec = df.query(f'seconds_in_bucket >= {second}').groupby('time_id').agg(aggregate_dictionary)
        df_feature_sec = df_feature_sec.reset_index()
        
        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns]
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))
        
        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    
    return df_feature

In [9]:
%%time
file_path = data_dir + "trade_train.parquet/stock_id=0"
preprocessor_trade(file_path)

CPU times: user 4.06 s, sys: 70.8 ms, total: 4.13 s
Wall time: 3.98 s


,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,row_id
0,0.002006,40,3179,2.750000,0.001308,21.0,1587.0,2.571429,0-5
1,0.000901,30,1289,1.900000,0.000587,16.0,900.0,2.250000,0-11
2,0.001961,25,2161,2.720000,0.001137,12.0,1189.0,3.166667,0-16
3,0.001561,15,1962,3.933333,0.001089,9.0,1556.0,5.111111,0-31
4,0.000871,22,1791,4.045455,0.000453,11.0,1219.0,4.909091,0-62
...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,3450,3.057692,0.001162,35.0,2365.0,3.257143,0-32751
3826,0.001411,28,4547,3.892857,0.001066,12.0,2161.0,4.250000,0-32753
3827,0.001521,36,4250,3.500000,0.001242,22.0,2294.0,3.727273,0-32758
3828,0.001794,53,3217,2.150943,0.001404,25.0,1627.0,1.920000,0-32763


## Combined preprocessor function

In [10]:
def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(preprocessor_book(file_path_book),preprocessor_trade(file_path_trade),on='row_id',how='left')
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df


In [11]:
list_stock_ids = [0,1]
preprocessor(list_stock_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   18.3s finished


,log_return_realized_volatility,log_return2_realized_volatility,log_return3_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,...,wap_mean_300,row_id,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0.004499,0.006999,0.006119,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,...,1.003753,0-5,0.002006,40,3179,2.750000,0.001308,21.0,1587.0,2.571429
1,0.001204,0.002476,0.002320,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,...,1.000397,0-11,0.000901,30,1289,1.900000,0.000587,16.0,900.0,2.250000
2,0.002369,0.004801,0.004684,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,...,0.998685,0-16,0.001961,25,2161,2.720000,0.001137,12.0,1189.0,3.166667
3,0.002574,0.003637,0.002881,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,...,0.998436,0-31,0.001561,15,1962,3.933333,0.001089,9.0,1556.0,5.111111
4,0.001894,0.003257,0.003166,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,...,0.999488,0-62,0.000871,22,1791,4.045455,0.000453,11.0,1219.0,4.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,0.003723,0.004996,0.004719,0.000330,0.000597,0.000157,-0.000118,125.013029,296.185668,1.000142,...,1.000130,1-32751,0.001776,49,3249,2.775510,0.001280,23.0,1889.0,3.608696
7656,0.010829,0.012168,0.011917,0.000403,0.000922,0.000159,-0.000125,254.006073,567.840081,1.007503,...,1.012343,1-32753,0.008492,183,75903,7.874317,0.006310,88.0,30858.0,8.136364
7657,0.003135,0.004268,0.004681,0.000243,0.000648,0.000141,-0.000132,163.645367,426.603834,1.000854,...,1.001250,1-32758,0.001927,26,2239,2.615385,0.001567,11.0,980.0,2.727273
7658,0.003750,0.005773,0.005369,0.000199,0.000421,0.000190,-0.000231,138.235023,526.317972,1.003032,...,1.004296,1-32763,0.002856,109,16648,2.935780,0.001919,57.0,8274.0,2.701754


## Training set

In [12]:
train = pd.read_csv(data_dir + 'train.csv')

In [13]:
train_ids = train.stock_id.unique()

In [14]:
%%time
df_train = preprocessor(list_stock_ids= train_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 14.2min finished


CPU times: user 3.32 s, sys: 255 ms, total: 3.58 s
Wall time: 14min 12s


In [15]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_train = train.merge(df_train, on = ['row_id'], how = 'left')

In [16]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return2_realized_volatility,log_return3_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,...,total_volume_mean_300,wap_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0-5,0.004136,0.004499,0.006999,0.006119,0.000388,0.000852,0.000176,-0.000151,134.894040,...,294.928058,1.003753,0.002006,40.0,3179.0,2.750000,0.001308,21.0,1587.0,2.571429
1,0-11,0.001445,0.001204,0.002476,0.002320,0.000212,0.000394,0.000142,-0.000135,142.050000,...,484.521739,1.000397,0.000901,30.0,1289.0,1.900000,0.000587,16.0,900.0,2.250000
2,0-16,0.002168,0.002369,0.004801,0.004684,0.000331,0.000725,0.000197,-0.000198,141.414894,...,455.235294,0.998685,0.001961,25.0,2161.0,2.720000,0.001137,12.0,1189.0,3.166667
3,0-31,0.002195,0.002574,0.003637,0.002881,0.000380,0.000860,0.000190,-0.000108,146.216667,...,418.169811,0.998436,0.001561,15.0,1962.0,3.933333,0.001089,9.0,1556.0,5.111111
4,0-62,0.001747,0.001894,0.003257,0.003166,0.000254,0.000397,0.000191,-0.000109,123.846591,...,407.584270,0.999488,0.000871,22.0,1791.0,4.045455,0.000453,11.0,1219.0,4.909091


In [17]:
df_train.isnull().sum()

row_id                                        0
target                                        0
log_return_realized_volatility                0
log_return2_realized_volatility               0
log_return3_realized_volatility               0
wap_balance_mean                              0
price_spread_mean                             0
bid_spread_mean                               0
ask_spread_mean                               0
volume_imbalance_mean                         0
total_volume_mean                             0
wap_mean                                      0
log_return_realized_volatility_300            0
log_return2_realized_volatility_300           0
log_return3_realized_volatility_300           0
wap_balance_mean_300                          0
price_spread_mean_300                         0
bid_spread_mean_300                           0
ask_spread_mean_300                           0
volume_imbalance_mean_300                     0
total_volume_mean_300                   

In [18]:
df_train.fillna(0, inplace=True)

In [19]:
df_train.isna().sum()

row_id                                      0
target                                      0
log_return_realized_volatility              0
log_return2_realized_volatility             0
log_return3_realized_volatility             0
wap_balance_mean                            0
price_spread_mean                           0
bid_spread_mean                             0
ask_spread_mean                             0
volume_imbalance_mean                       0
total_volume_mean                           0
wap_mean                                    0
log_return_realized_volatility_300          0
log_return2_realized_volatility_300         0
log_return3_realized_volatility_300         0
wap_balance_mean_300                        0
price_spread_mean_300                       0
bid_spread_mean_300                         0
ask_spread_mean_300                         0
volume_imbalance_mean_300                   0
total_volume_mean_300                       0
wap_mean_300                      

## Test set

In [20]:
test = pd.read_csv(data_dir + 'test.csv')

In [21]:
test_ids = test.stock_id.unique()

In [22]:
%%time
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)

CPU times: user 20 ms, sys: 57 µs, total: 20 ms
Wall time: 200 ms


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished


In [23]:
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

## Target encoding by stock_id

In [24]:
from sklearn.model_selection import KFold
#stock_id target encoding
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean() 
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean) # test_set

#training
tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits = 20, shuffle=True,random_state = 19911109)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()

    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

## Model Building

## LSTM-Regression

In [25]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [26]:
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [27]:
X= df_train.drop(['row_id','target'],axis=1)
y= df_train['target']

In [28]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=25, random_state=19901028, shuffle=True)
oof = pd.DataFrame()                 # out-of-fold result
models = []                          # models
scores = 0.0   

In [29]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    
    # create dataset
    X_train, y_train = X.loc[trn_idx], y[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    

CPU times: user 1.91 s, sys: 227 ms, total: 2.14 s
Wall time: 2.14 s


In [30]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    
    # create dataset
    X_train, y_train = X.loc[trn_idx], y[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    


CPU times: user 1.94 s, sys: 195 ms, total: 2.14 s
Wall time: 2.14 s


In [31]:
from sklearn.preprocessing import StandardScaler

In [32]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

**Labeling

In [33]:
n = 30  
Xtrain = []
ytrain = []

for i in range(n, len(X_train_scaled)): 
    Xtrain.append(X_train_scaled[(i - n):i, :X_train_scaled.shape[1]])
    ytrain.append(y_train[i-1:i]) # predict the next record


In [34]:
Xtrain, ytrain = (np.array(Xtrain), np.array(ytrain))
#Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[2]))

In [35]:
Xtrain[0].shape

(30, 30)

In [36]:
Xvalid = []
yvalid = []

In [37]:
m = 30
for i in range(m, len(X_valid)): #use different numbers
    Xvalid.append(X_valid_scaled[(i - n):i, :X_valid_scaled.shape[1]])
    yvalid.append(y_valid[i-1:i]) # predict the next record

In [38]:
Xvalid, yvalid = (np.array(Xvalid), np.array(yvalid))
#Xvalid = np.reshape(Xvalid, (Xvalid.shape[0], Xvalid.shape[1], Xvalid.shape[2]))

In [39]:
yvalid.shape

(17127, 1)

In [40]:
def layer_maker(n_layers, n_nodes, activation, drop=None, d_rate=.5):
    """
    Creates a specified number of hidden layers for an RNN
    Optional: Adds regularization option - the dropout layer to prevent potential overfitting (if necessary)
    """
    
    # Creating the specified number of hidden layers with the specified number of nodes
    for x in range(1,n_layers+1):
        model.add(LSTM(n_nodes, activation=activation, return_sequences=True))

        # Adds a Dropout layer after every Nth hidden layer (the 'drop' variable)
        try:
            if x % drop == 0:
                model.add(Dropout(d_rate))
        except:
            pass

In [41]:
# Instatiating the model
model = Sequential()

# Activation
activ = "tanh"

# Input layer
model.add(LSTM(50, 
               activation=activ, 
               return_sequences=True, 
               input_shape=(30, Xtrain.shape[1])))

# Hidden layers
layer_maker(n_layers=2, 
        n_nodes=30, 
            activation=activ,
            drop=1,
            d_rate=.1)

# Final Hidden layer
model.add(LSTM(30, activation=activ))

# Output layer
model.add(Dense(1))

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 50)            16200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 30)            9720      
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 30)            7320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dense (Dense)                (None, 1)                 31        
Total params: 40,591
Trainable params: 40,591
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])

In [43]:
LSTM_Model = model.fit(Xtrain, ytrain, epochs = 20, batch_size =32, validation_split=0.1)

Epoch 1/20
11581/11581 [==============================] - 650s 55ms/step - loss: 1.8106e-05 - mean_squared_error: 1.8106e-05 - val_loss: 1.5497e-06 - val_mean_squared_error: 1.5497e-06
Epoch 2/20
11581/11581 [==============================] - 647s 56ms/step - loss: 1.8683e-06 - mean_squared_error: 1.8683e-06 - val_loss: 1.4068e-06 - val_mean_squared_error: 1.4068e-06
Epoch 3/20
11581/11581 [==============================] - 641s 55ms/step - loss: 1.7519e-06 - mean_squared_error: 1.7519e-06 - val_loss: 1.6227e-06 - val_mean_squared_error: 1.6227e-06
Epoch 4/20
11581/11581 [==============================] - 644s 56ms/step - loss: 1.6844e-06 - mean_squared_error: 1.6844e-06 - val_loss: 1.4657e-06 - val_mean_squared_error: 1.4657e-06
Epoch 5/20
11581/11581 [==============================] - 651s 56ms/step - loss: 1.6764e-06 - mean_squared_error: 1.6764e-06 - val_loss: 1.2488e-06 - val_mean_squared_error: 1.2488e-06
Epoch 6/20
11581/11581 [==============================] - 649s 56ms/step - 

In [44]:
#validation
y_pred = model.predict(Xvalid)

In [45]:
y_pred.shape

(17127, 1)

In [46]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

In [47]:
RMSPE = round(rmspe(y_true = yvalid, y_pred = y_pred),3)
print(f'Performance of the　prediction: , RMSPE: {RMSPE}')
    
 

Performance of the　prediction: , RMSPE: 0.252


In [48]:
 

    #keep scores and models
    scores += RMSPE / 25
    models.append(model)
    print("*" * 100)

****************************************************************************************************
